In [1]:
from willitfit.scrapers.IKEA import scrape_product
scrape_product('20450520')



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/home/flo/.wdm/drivers/chromedriver/linux64/91.0.4472.19/chromedriver] found in cache


<div aria-labelledby="product-details-packaging" class="range-revamp-accordion__content" id="SEC_product-details-packaging" role="region"><div><div class="range-revamp-product-details__container"><span class="range-revamp-product-details__header notranslate">VINDKAST</span><span>Hängeleuchte</span><span class="range-revamp-product-details__label">Artikelnummer:</span><span class="range-revamp-product-identifier" data-ga-action="article_number_click" data-ga-label="details_204.505.20" x-ms-format-detection="none"><span class="range-revamp-product-identifier__value">204.505.20</span></span><div class="range-revamp-product-details__container"><span class="range-revamp-product-details__label">Breite: 43 cm</span><span class="range-revamp-product-details__label">Höhe: 10 cm</span><span class="range-revamp-product-details__label">Länge: 47 cm</span><span class="range-revamp-product-details__label">Gewicht: 0.98 kg</span><span class="range-revamp-product-details__label">Paket(e): 1</span></di

In [2]:
from willitfit.scrapers.IKEA import product_info_and_update_csv_database

article_code = {'10364740':1}





product_info_and_update_csv_database(article_code)
















====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/home/flo/.wdm/drivers/chromedriver/linux64/91.0.4472.19/chromedriver] found in cache


[['10364740', 1, [(1, 13, 25, 50, 1.2)]]]

In [3]:
import pandas as pd
pd.read_csv('/home/kasia/code/willitfit/willitfit/data/ikea_database/ikea_database.csv',index_col=[0])









FileNotFoundError: [Errno 2] No such file or directory: '/home/kasia/code/willitfit/willitfit/data/ikea_database/ikea_database.csv'

In [ ]:
import requests
import os
country_domain= 'de'
website_language = 'de'
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium import webdriver





def chrome_settings():
    """
    """
    
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument('--window-size=1920x1080') 
    
    return chrome_options


IKEA_URL = f"https://www.ikea.com/{country_domain}/{website_language}/"
IKEA_SEARCH_URL = f"search/products/?q="
#r = requests.get(os.path.join(IKEA_URL,IKEA_SEARCH_URL,'xxx'))


driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_settings())
driver.get(os.path.join(IKEA_URL,IKEA_SEARCH_URL,'xxx'))
#important_part_of_page = driver.find_element_by_class_name('results__list')
#tag = important_part_of_page.find_element_by_tag_name('a')



In [ ]:
important_part_of_page = driver.find_element_by_class_name('results__list')
tag = important_part_of_page.find_element_by_tag_name('a')





In [ ]:
try:
    important_part_of_page.find_element_by_tag_name('a')

except:
    print('gsdgsja')
    
    